# Meter Extraction
Goal: Extract distinct meters and relevant attributes from PVACD Access DB

In [1]:
import pyodbc
import pandas as pd
import numpy as np

### Load meter types and mapping from CSV
This is used to assign each imported meter a meter type ID

In [2]:
#Load CSVs from the data folder
metertypes = pd.read_csv('./api/data/devdata_metertypes.csv')
oldmetertypes = pd.read_csv('./api/data/devdata_metertypemap.csv')

In [3]:
#Merge metertypes and metermap based on brand and model_number
metertypemap = oldmetertypes.merge(metertypes, left_on=['sBrand', 'newModel'], right_on=['brand','model_number'], how='left')
metertypemap.head()

,sSize,sBrand,sModel,newModel,id,brand,series,model_number,size,description
0,NaN,NaN,"1""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8.0,NaN,308,MO308,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,BADGER,"2""","2""",NaN,NaN,NaN,NaN,NaN,NaN
3,8.0,BADGER,"1""","1""",NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,BADGER,"1""","1""",NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
metertypemap.head(30)

,sSize,sBrand,sModel,newModel,id,brand,series,model_number,size,description
0,NaN,NaN,"1""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8.000,NaN,308,MO308,NaN,NaN,NaN,NaN,NaN,NaN
2,2.000,BADGER,"2""","2""",NaN,NaN,NaN,NaN,NaN,NaN
3,8.000,BADGER,"1""","1""",NaN,NaN,NaN,NaN,NaN,NaN
4,1.000,BADGER,"1""","1""",NaN,NaN,NaN,NaN,NaN,NaN
5,1.500,BADGER,1.5,T-0160,21.0,BADGER,NaN,T-0160,1.500,1.5 TURBO W/BOLT KITS & FLGS
6,3.000,BADGER,"3""","3""",18.0,BADGER,NaN,"3""",3.000,"3"" CHAMBER"
7,3.000,BADGER,3IN,"3""",18.0,BADGER,NaN,"3""",3.000,"3"" CHAMBER"
8,6.000,BADGER,3/4 IN,"3/4""",19.0,BADGER,NaN,"3/4""",0.750,BADGER W / CLPGS
9,4.000,BADGER,5/8,"5/8""",20.0,BADGER,NaN,"5/8""",0.625,BADGER W / CLPGS


In [5]:
#Fix badger turbo ids
metertypemap.loc[22,'id'] = 22
metertypemap.loc[23,'id'] = 21
metertypemap.loc[24,'id'] = 21
metertypemap.loc[25,'id'] = 23

There are a few type ids that still are not mapped (see below). The ' Badger' issue is fixed in the function. 'Other' and Nan brands are ignored, so some meters will end up without IDs.

In [ ]:
metertypemap[metertypemap.id.isna()]

In [6]:
misspell = {' BADGER':'BADGER','MCROMETER':'MCCROMETER'}
oldmeters = {'ROCKWELL':28,'SENSUS':29,'SPARLIN':30,'MASTER METER':31,'COGNITIS FLUID DATA':32,'FLOW CONTROL':33}

# A function that maps an input size, brand, and model to the meter type ID
def getMeterType(size, brand, model):
    if brand in misspell.keys():
        brand = misspell[brand]

    #Immediately assign old brands
    if brand in oldmeters.keys():
        return oldmeters[brand]

    #Additional mappings for empty or missing model numbers
    if model is None or model == '':
        if brand == 'MCCROMETER' and size is not None:
            model = 'MO3' + str(int(size)).zfill(2)
        elif brand == 'BADGER' and size is not None:
            model = str(int(size)) + '"'
        
    #print(model)

    # Filter the meter type map to only include the size, brand, and model number
    filtered = metertypemap[(metertypemap['sSize'] == size) & (metertypemap['sBrand'] == brand) & (metertypemap['sModel'] == model)]
    # If there is a match, return the meter type ID
    if len(filtered) > 0:
        return filtered['id'].values[0]
    # Otherwise, return None
    else:
        return None

In [ ]:
#Test function
print(getMeterType(2, ' BADGER', '2"'))
print(getMeterType(6, 'MCROMETER', '308'))
print(getMeterType(1.5, 'BADGER TURBO', '1.5'))
print(getMeterType(4.0,'MCCROMETER',None))
print(getMeterType(5,'SENSUS',None))

### Data Import
Load organizations from CSV and get the meters access DB.

In [7]:
#Set up PyODBC connection to Access database
conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=C:\Users\clmbn\NMWaterData\PVACDdb\PVACD AMS Jan2023.accdb;'
    )
cnxn = pyodbc.connect(conn_str)

In [8]:
#Import landowners csv
orgs = pd.read_csv('./api/data/landowners.csv')
orgs.head()

,organization_name,address,city,state,zip,phone,mobile,note,id
0,AG SERVICES,1905 OLD DEXTER HIGHWAY,ROSWELL,NM,88203,575-626-4939,NaN,NEAL POPE AND RAY POPE,1
1,AIRGAS,1103 EAST SECOND ST,ROSWELL,NM,88201,575-622-1950,NaN,NaN,2
2,"AMERICAN OXYGEN CO., INC.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,"ARMSTRONG & SON, LLC",P.O. BOX 1973,ROSWELL,NM,88202,575-622-1080,NaN,NaN,4
4,ARTESIA COUNTRY CLUB,P.O. BOX 1305,ARTESIA,NM,88211,575-746-2055,NaN,NaN,5


#### Meters
There are two tables: tblMeterX and and tblMeter. It appears that tblMeterX is mostly tracking maintenance while tblMeter is mostly unique meter serial numbers.
I will just import from tblMeter.

Fields:
- sTag, sSize, sBrand, sContact, sSerialNumber, sRANumber, sTRSS, sLatLong, sModel, sStatus

In [9]:
qry = '''
    SELECT DISTINCT sSerialNumber, sSize, sBrand, sModel, sContact, sTag, sRANumber, sTRSS, sLatLong, sStatus 
    FROM tblMeter
    WHERE sSerialNumber IS NOT NULL AND sSerialNumber <> ''
'''
meter_df = pd.read_sql_query(qry, cnxn)
meter_df.head(20)

C:\Users\clmbn\AppData\Local\Temp\ipykernel_6904\978414569.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  meter_df = pd.read_sql_query(qry, cnxn)


,sSerialNumber,sSize,sBrand,sModel,sContact,sTag,sRANumber,sTRSS,sLatLong,sStatus
0,88-8-817,8.0,MCCROMETER,308,HALDEMAN W.T. ESTATE,None,763;1180-S,17.26.14.133,N32* 50' 08.9'' W104* 21' 38.0'',S
1,94-8-2251,8.0,MCCROMETER,308,DAVID AND TRACEY CORN,221C4,2417 POD 2,8.24.28.413,,I
2,00-10-6922,10.0,MCCROMETER,310,AG SERVICES,None,,"ROSWELL, NM",,S
3,00-10-6923,10.0,MCCROMETER,310,N.M.S.U. EXPER STAT.,20138,827,18.26.16.211,N32*-45' 16.4'' W104*-23' 11.1'',I
4,00-10-6924,10.0,MCCROMETER,310,WAIDE FARMS,None,None,None,None,S
5,00-10-6925,10.0,MCCROMETER,310,JULIUS ROBERSON/P.V.PUMP,None,,CARLSBAD NM,,S
6,00-10-6926,10.0,MCCROMETER,None,MIKE CARRASCO,None,None,CARLSBAD NM,None,S
7,00-10-6927,10.0,MCCROMETER,310,BOGLE FARMS,20900,410-S,13.26.30.133,N33*-09' 46.1'' W104*-23' 13.5'',I
8,00-10-6928,10.0,MCCROMETER,310,BROWN BYRON,2004E,2769,11.23.22.334,N33* 20' 08.6'' W104* 36' 30.6'',I
9,00-10-6929,10.0,MCCROMETER,310,PECOS VALLEY PUMP,None,None,ARTESIA,None,S


In [ ]:
meter_df.tail()

### Prepare data for new database

In [10]:
#Merge in contact id from contact_df
meters = meter_df.merge(orgs[['organization_name', 'id']], left_on='sContact', right_on='organization_name', how='left')
meters.head(20)

,sSerialNumber,sSize,sBrand,sModel,sContact,sTag,sRANumber,sTRSS,sLatLong,sStatus,organization_name,id
0,88-8-817,8.0,MCCROMETER,308,HALDEMAN W.T. ESTATE,None,763;1180-S,17.26.14.133,N32* 50' 08.9'' W104* 21' 38.0'',S,NaN,NaN
1,94-8-2251,8.0,MCCROMETER,308,DAVID AND TRACEY CORN,221C4,2417 POD 2,8.24.28.413,,I,DAVID AND TRACEY CORN,53.0
2,00-10-6922,10.0,MCCROMETER,310,AG SERVICES,None,,"ROSWELL, NM",,S,AG SERVICES,1.0
3,00-10-6923,10.0,MCCROMETER,310,N.M.S.U. EXPER STAT.,20138,827,18.26.16.211,N32*-45' 16.4'' W104*-23' 11.1'',I,NaN,NaN
4,00-10-6924,10.0,MCCROMETER,310,WAIDE FARMS,None,None,None,None,S,NaN,NaN
5,00-10-6925,10.0,MCCROMETER,310,JULIUS ROBERSON/P.V.PUMP,None,,CARLSBAD NM,,S,NaN,NaN
6,00-10-6926,10.0,MCCROMETER,None,MIKE CARRASCO,None,None,CARLSBAD NM,None,S,NaN,NaN
7,00-10-6927,10.0,MCCROMETER,310,BOGLE FARMS,20900,410-S,13.26.30.133,N33*-09' 46.1'' W104*-23' 13.5'',I,NaN,NaN
8,00-10-6928,10.0,MCCROMETER,310,BROWN BYRON,2004E,2769,11.23.22.334,N33* 20' 08.6'' W104* 36' 30.6'',I,NaN,NaN
9,00-10-6929,10.0,MCCROMETER,310,PECOS VALLEY PUMP,None,None,ARTESIA,None,S,PECOS VALLEY PUMP,160.0


In [11]:
#Assign meters new meter type id based on getMeterType function
meters['meter_type_id'] = meter_df.apply(lambda x: getMeterType(x['sSize'], x['sBrand'], x['sModel']), axis=1)

In [ ]:
meters.head(20)

In [12]:
#Assess meters with no meter type id and status of S
meters[(meters['meter_type_id'].isna()) & (meters['sStatus'] == 'I')]


,sSerialNumber,sSize,sBrand,sModel,sContact,sTag,sRANumber,sTRSS,sLatLong,sStatus,organization_name,id,meter_type_id
799,10016992,1.5,BADGER,,BILLBERRY JOHN,None,8134-S,16.25.1 LOT 15,N32* 57' 18.3'' W104* 26' 09.1'',I,NaN,NaN,NaN
800,10016993,1.5,BADGER,,BILBERRY JOHN,20A8A,8134,16.25.1.LOT 15,N32* 57' 18.1'' W104* 26' 08.4'',I,NaN,NaN,NaN
876,11207024,2.0,BADGER,"2""",RYAN & NICOLE MILLER,2086A,11337,10.24.8.312,N33* 27' 31.1'' W104* 32' 14.2'',I,NaN,NaN,NaN
1875,211567746,8.0,BADGER,"1""",HARVEY LEON,None,7385,12.25.8.343,None,I,NaN,NaN,NaN
2946,853121,0.0,OTHER,0,"HAGERMAN, TOWN OF",None,0,0,None,I,NaN,NaN,NaN


I finally have to just manually fix a few rows. I should verify I am fixing the correct rows when rerunning this as the data loaded by the query could change.

In [13]:
# Some final manual id assignments
meters.loc[799,'meter_type_id'] = 21
meters.loc[800,'meter_type_id'] = 21
meters.loc[876,'meter_type_id'] = 17
meters.loc[1875,'meter_type_id'] = 16


In [ ]:
meters.tail()

In [14]:
cnxn.close()

Assign a meter status code ID

In [15]:
meter_status_codes = {'X':3,'I':1,'S':5,'R':6,'A':6,'N':2}
#Assign status id based on meter status code
meters['meter_status_id'] = meters.apply(lambda x: meter_status_codes.get(x['sStatus'],6), axis=1)
meters.head()

,sSerialNumber,sSize,sBrand,sModel,sContact,sTag,sRANumber,sTRSS,sLatLong,sStatus,organization_name,id,meter_type_id,meter_status_id
0,88-8-817,8.0,MCCROMETER,308,HALDEMAN W.T. ESTATE,None,763;1180-S,17.26.14.133,N32* 50' 08.9'' W104* 21' 38.0'',S,NaN,NaN,3.0,5
1,94-8-2251,8.0,MCCROMETER,308,DAVID AND TRACEY CORN,221C4,2417 POD 2,8.24.28.413,,I,DAVID AND TRACEY CORN,53.0,3.0,1
2,00-10-6922,10.0,MCCROMETER,310,AG SERVICES,None,,"ROSWELL, NM",,S,AG SERVICES,1.0,4.0,5
3,00-10-6923,10.0,MCCROMETER,310,N.M.S.U. EXPER STAT.,20138,827,18.26.16.211,N32*-45' 16.4'' W104*-23' 11.1'',I,NaN,NaN,4.0,1
4,00-10-6924,10.0,MCCROMETER,310,WAIDE FARMS,None,None,None,None,S,NaN,NaN,4.0,5


In [ ]:
meters.tail(30)

### Parse lat/long
Extract a decimal lat and long from strings in sLatLong and load to separate columns

In [16]:
import re

In [17]:
latlong_pattern = re.compile(r"[N,W](\d+)\*(\d+)'(\d+.\d*)\"")
def getLatLong(sLatLong):
    #print(sLatLong)

    if sLatLong is None or sLatLong == '':
        return (None, None)
    
    #Strip any whitespace and '-' from string
    sLatLong = sLatLong.replace(' ','').replace('-','')

    #Replace '' with "
    sLatLong = sLatLong.replace('\'\'','"')

    #Find patterns, should be two matchs (N and W) and groups for each (degrees, minutes, seconds)
    matches = latlong_pattern.findall(sLatLong)

    #For each match, convert groups to decimal degrees
    if len(matches) == 2:
        lat = float(matches[0][0]) + float(matches[0][1])/60 + float(matches[0][2])/3600
        long = float(matches[1][0]) + float(matches[1][1])/60 + float(matches[1][2])/3600
        return lat, long
    else:
        return (None, None)

In [ ]:
#Test function with typical string
teststr1 = "N32*-45\' 16.4\" W106*-4\' 06.4\""
teststr2 = "N33*-06.758 W106*-4\' 06.4\""
teststr3 = "N32* 50' 08.9''  W104* 21' 38.0''"
print(getLatLong(teststr1))
print(getLatLong(teststr2))
print(getLatLong(teststr3))

In [18]:
# Create latitude and longitude columns using getLatLong function on sLatLong column
meters['latitude'] = meters.apply(lambda x: getLatLong(x['sLatLong'])[0], axis=1)
meters['longitude'] = meters.apply(lambda x: getLatLong(x['sLatLong'])[1], axis=1)
meters.head()

,sSerialNumber,sSize,sBrand,sModel,sContact,sTag,sRANumber,sTRSS,sLatLong,sStatus,organization_name,id,meter_type_id,meter_status_id,latitude,longitude
0,88-8-817,8.0,MCCROMETER,308,HALDEMAN W.T. ESTATE,None,763;1180-S,17.26.14.133,N32* 50' 08.9'' W104* 21' 38.0'',S,NaN,NaN,3.0,5,32.835806,104.360556
1,94-8-2251,8.0,MCCROMETER,308,DAVID AND TRACEY CORN,221C4,2417 POD 2,8.24.28.413,,I,DAVID AND TRACEY CORN,53.0,3.0,1,NaN,NaN
2,00-10-6922,10.0,MCCROMETER,310,AG SERVICES,None,,"ROSWELL, NM",,S,AG SERVICES,1.0,4.0,5,NaN,NaN
3,00-10-6923,10.0,MCCROMETER,310,N.M.S.U. EXPER STAT.,20138,827,18.26.16.211,N32*-45' 16.4'' W104*-23' 11.1'',I,NaN,NaN,4.0,1,32.754556,104.386417
4,00-10-6924,10.0,MCCROMETER,310,WAIDE FARMS,None,None,None,None,S,NaN,NaN,4.0,5,NaN,NaN


In [19]:
#Assess number of rows with non NA latitude and longitude
len(meters[meters['latitude'].notna()])

1435

In [20]:
len(meters)

4509

In [21]:
#Drop duplicate serial numbers from meter_df
meters.drop_duplicates(subset='sSerialNumber', keep='first', inplace=True)
len(meters)

4499

In [23]:
meters.head()

,sSerialNumber,sSize,sBrand,sModel,sContact,sTag,sRANumber,sTRSS,sLatLong,sStatus,organization_name,id,meter_type_id,meter_status_id,latitude,longitude
0,88-8-817,8.0,MCCROMETER,308,HALDEMAN W.T. ESTATE,None,763;1180-S,17.26.14.133,N32* 50' 08.9'' W104* 21' 38.0'',S,NaN,NaN,3.0,5,32.835806,104.360556
1,94-8-2251,8.0,MCCROMETER,308,DAVID AND TRACEY CORN,221C4,2417 POD 2,8.24.28.413,,I,DAVID AND TRACEY CORN,53.0,3.0,1,NaN,NaN
2,00-10-6922,10.0,MCCROMETER,310,AG SERVICES,None,,"ROSWELL, NM",,S,AG SERVICES,1.0,4.0,5,NaN,NaN
3,00-10-6923,10.0,MCCROMETER,310,N.M.S.U. EXPER STAT.,20138,827,18.26.16.211,N32*-45' 16.4'' W104*-23' 11.1'',I,NaN,NaN,4.0,1,32.754556,104.386417
4,00-10-6924,10.0,MCCROMETER,310,WAIDE FARMS,None,None,None,None,S,NaN,NaN,4.0,5,NaN,NaN


Data extraction and classification is done. Now I will focus on exporting just the important columns.

In [24]:
#Drop columns sSize, sBrand, sModel, sLatLong, sStatus, organization_name
meters.drop(columns=['sSize', 'sBrand', 'sModel', 'sLatLong', 'sStatus', 'organization_name'], inplace=True)
meters.head()

,sSerialNumber,sContact,sTag,sRANumber,sTRSS,id,meter_type_id,meter_status_id,latitude,longitude
0,88-8-817,HALDEMAN W.T. ESTATE,None,763;1180-S,17.26.14.133,NaN,3.0,5,32.835806,104.360556
1,94-8-2251,DAVID AND TRACEY CORN,221C4,2417 POD 2,8.24.28.413,53.0,3.0,1,NaN,NaN
2,00-10-6922,AG SERVICES,None,,"ROSWELL, NM",1.0,4.0,5,NaN,NaN
3,00-10-6923,N.M.S.U. EXPER STAT.,20138,827,18.26.16.211,NaN,4.0,1,32.754556,104.386417
4,00-10-6924,WAIDE FARMS,None,None,None,NaN,4.0,5,NaN,NaN


In [25]:
#Rename id to owner_id
meters.rename(columns={'id':'owner_id'}, inplace=True)

In [28]:
# all ids should be integers
meters['owner_id'] = meters['owner_id'].astype("Int64")
meters['meter_type_id'] = meters['meter_type_id'].astype("Int64")
meters.head()

,sSerialNumber,sContact,sTag,sRANumber,sTRSS,owner_id,meter_type_id,meter_status_id,latitude,longitude
0,88-8-817,HALDEMAN W.T. ESTATE,None,763;1180-S,17.26.14.133,<NA>,3,5,32.835806,104.360556
1,94-8-2251,DAVID AND TRACEY CORN,221C4,2417 POD 2,8.24.28.413,53,3,1,NaN,NaN
2,00-10-6922,AG SERVICES,None,,"ROSWELL, NM",1,4,5,NaN,NaN
3,00-10-6923,N.M.S.U. EXPER STAT.,20138,827,18.26.16.211,<NA>,4,1,32.754556,104.386417
4,00-10-6924,WAIDE FARMS,None,None,None,<NA>,4,5,NaN,NaN


At this point, the data is ready to be split into more specific tables. This is better handled in a new notebook as the process may change. Save the data to a CSV for now.

In [29]:
#Save to CSV
meters.to_csv('./api/data/extracted_meters.csv', index=False)